In [52]:
# For Python 2 / 3 compatability
from __future__ import print_function

In [53]:
training_data = [
    
     ['2','yes','pass'],
     ['5','no','fail'],
     ['6','yes','pass'],
     
     ['3','yes','fail'],
     ['2','no','fail'],
    
 
]

In [54]:

header = [  "noOfHrStudy", "referenceStudyMaterialUsed","passOrfail"]

In [55]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [56]:
unique_vals(training_data, 0)

{'2', '3', '5', '6'}

In [57]:
unique_vals(training_data, 1)

{'no', 'yes'}

In [58]:
unique_vals(training_data, 2)

{'fail', 'pass'}

In [59]:
unique_vals(training_data,1)

{'no', 'yes'}

In [60]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [61]:
#######
# Demo:
class_counts(training_data)
#######

{'pass': 2, 'fail': 3}

In [62]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [63]:
is_numeric(7)

True

In [64]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [65]:
q=Question(1, 'Sports')
q

Is referenceStudyMaterialUsed == Sports?

In [66]:
example = training_data[0]
q.match(example)

False

In [67]:
qq1=Question(0, '4')
qq1


Is noOfHrStudy == 4?

In [68]:
r1=Question(2,'yes')
r1

Is passOrfail == yes?

In [69]:
example1 = training_data[0]
r1.match(example1)

False

In [70]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [71]:
true_rows, false_rows = partition(training_data, Question(1, 'yes'))
true_rows

[['2', 'yes', 'pass'], ['6', 'yes', 'pass'], ['3', 'yes', 'fail']]

In [72]:
false_rows

[['5', 'no', 'fail'], ['2', 'no', 'fail']]

In [73]:
def gini(rows):

    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [74]:
no_mixing = [['Yes'],
              ['Yes']]
gini(no_mixing)

0.0

In [75]:
no_mixing = [['Yes'],
              ['No']]
gini(no_mixing)

0.5

In [76]:
def info_gain(left, right, current_uncertainty):
   
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [77]:

current_uncertainty = gini(training_data)
current_uncertainty

0.48

In [78]:
# How much information do we gain by partioning on 'Red'?
true_rows, false_rows = partition(training_data, Question(0, '9'))
info_gain(true_rows, false_rows, current_uncertainty)

0.0

In [79]:
# What about if we partioned on 'Yellow' instead?
true_rows, false_rows = partition(training_data, Question(0,'6'))
info_gain(true_rows, false_rows, current_uncertainty)

0.17999999999999994

In [80]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [81]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is referenceStudyMaterialUsed == no?

In [82]:
class Leaf:
  

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [83]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [84]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [85]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [86]:
my_tree = build_tree(training_data)

In [87]:
print_tree(my_tree)

Is referenceStudyMaterialUsed == no?
--> True:
  Predict {'fail': 2}
--> False:
  Is noOfHrStudy == 3?
  --> True:
    Predict {'fail': 1}
  --> False:
    Predict {'pass': 2}
